In [5]:
import numpy as np


test_data_raw = (
"""#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#"""
)


def parse_data(data):
    return [
        np.array([list(row) for row in matrix.splitlines()])
        for matrix in data.split("\n\n")
    ]


test_data = parse_data(test_data_raw)
test_data

[array([['#', '.', '#', '#', '.', '.', '#', '#', '.'],
        ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
        ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
        ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '.'],
        ['#', '.', '#', '.', '#', '#', '.', '#', '.']], dtype='<U1'),
 array([['#', '.', '.', '.', '#', '#', '.', '.', '#'],
        ['#', '.', '.', '.', '.', '#', '.', '.', '#'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '#'],
        ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
        ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '#'],
        ['#', '.', '.', '.', '.', '#', '.', '.', '#']], dtype='<U1')]

In [82]:
def find_reflection(matrix, axis=0):
    for i in range(1, matrix.shape[axis] // 2 + 2):
        size = min(i, matrix.shape[axis] - i)
        start = i - size
        mid = i
        end = i + size
        first = matrix.take(range(start, mid), axis=axis)
        second = matrix.take(range(end - 1, mid - 1, -1), axis=axis)
        if np.all(first == second):
            return i
    return None

print(find_reflection(test_data[0], axis=1))
print(find_reflection(test_data[1], axis=0))

5
4


In [84]:
def part1(data):
    result = [0, 0]
    for matrix in data:
        for axis in range(2):
            reflection = find_reflection(matrix, axis=axis)
            if reflection is not None:
                result[axis] += reflection
    return result[0] * 100 + result[1]


part1(test_data)

405

In [85]:
with open("input.txt") as f:
    data = parse_data(f.read())

data[0]

array([['.', '.', '.', '.', '.', '.', '.', '#', '.', '#', '#', '#', '#'],
       ['#', '.', '.', '#', '.', '#', '#', '.', '#', '#', '#', '#', '#'],
       ['.', '#', '.', '.', '.', '#', '.', '.', '#', '.', '.', '.', '.'],
       ['#', '.', '.', '#', '.', '#', '#', '#', '.', '.', '.', '.', '.'],
       ['.', '#', '#', '.', '#', '.', '.', '#', '#', '.', '#', '#', '.'],
       ['#', '#', '#', '#', '#', '.', '#', '.', '.', '#', '#', '#', '#'],
       ['.', '.', '.', '.', '#', '#', '#', '#', '#', '#', '.', '.', '#']],
      dtype='<U1')

In [87]:
part1(data)

8586